## **Detcting bifurcations using graph created with MST**

The following code takes the centerline points extracted by Materialise Mimics in .txt format as input, converts them to graph nodes, and links them using the Minimum Spanning Tree algorithm.

It also identifies the graph's branches and assigns different labels to them by finding supersegments. It is one of the firts approaches tried, it does not provide the final labeling.

It uses a data frame to store the information.

### **Imports**

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
import plotly.graph_objects as go
import networkx as nx
from scipy.spatial.distance import cdist
from scipy.spatial import distance_matrix

### **Google Drive connection**

In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### **Upload files**

##### Normal 1

In [3]:
file_path_rca_n1= '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Normal_1/rca_centerline.txt'
file_path_lca_n1= '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Normal_1/lca_centerline.txt'

##### Normal 7

In [ ]:
file_path_rca_n7 = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Normal_7/rca_centerline.txt'
file_path_lca_n7 = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Normal_7/lca_centerline.txt'

##### Diseased 7

In [ ]:
file_path_rca_d7 = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Diseased_7/rca_centerline.txt'
file_path_lca_d7 = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Diseased_7/lca_centerline.txt'

##### Diseased 9

In [ ]:
file_path_rca_d9 = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Diseased_9/rca_centerline.txt'
file_path_lca_d9 = '/content/drive/Shared drives/TFGs Coronarias 2024_25/Maren/Data/ASOCA 4 Cases Mimics/Diseased_9/lca_centerline.txt'

### **Functions**

In [4]:
def parse_and_export_centerlines(file_path):
    branches = defaultdict(list)  # Diccionario donde la clave es el ID de la rama
    current_branch = None  # Guarda el ID de la rama actual
    has_branches = False  # Bandera para detectar si hay múltiples ramas

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            # Detectar una nueva rama
            branch_match = re.match(r"\[New Branch Set\] Branch Segment (\d+):", line)
            if branch_match:
                current_branch = int(branch_match.group(1))
                has_branches = True  # Confirmamos que hay múltiples ramas
                continue

            # Si no hay identificadores de rama, asignamos un ID por defecto
            if current_branch is None:
                current_branch = 0

            # Extraer puntos de la rama actual (ignorar líneas vacías o encabezados)
            if re.match(r"^\s*-?\d+\.\d+", line):
                data = []
                for value in line.split():
                    try:
                        data.append(float(value))  # Convertir a float
                    except ValueError:
                        data.append(None)  # Manejar valores no numéricos como None
                branches[current_branch].append(data)

    # Crear lista de datos para el DataFrame
    all_data = []
    for branch_id, points in branches.items():
        for point in points:
            all_data.append([branch_id] + point)

    # Columnas según el formato esperado
    columns = ["Branch ID", "Px", "Py", "Pz", "Tx", "Ty", "Tz", "Nx", "Ny", "Nz",
               "BNx", "BNy", "BNz", "Dfit", "Dmin", "Dmax", "C", "Dh", "Xh", "Scf", "Area", "E"]

    # Crear DataFrame y retornarlo
    df = pd.DataFrame(all_data, columns=columns)

    return df

In [5]:
def visualize_graph(G, title='Coronary Artey Graph'):
    # Obtener las posiciones de los nodos
    pos = nx.get_node_attributes(G, 'pos')

    # Check if pos is empty
    if not pos:
        print("No positions found in the graph. Check if nodes have 'pos' attribute.")
        return

    # Crear listas para nodos y aristas
    node_x = []
    node_y = []
    node_z = []
    for p in pos.values():
        node_x.append(p[0])
        node_y.append(p[1])
        node_z.append(p[2])

    edge_x = []
    edge_y = []
    edge_z = []
    for edge in G.edges():
        # Check if node IDs are in pos before accessing them
        if edge[0] in pos and edge[1] in pos:
            x0, y0, z0 = pos[edge[0]]
            x1, y1, z1 = pos[edge[1]]
            edge_x.extend([x0, x1, None])
            edge_y.extend([y0, y1, None])
            edge_z.extend([z0, z1, None])
        else:
            print(f"Skipping edge {edge} as one or both nodes are missing position data.")

    # Crear la visualización interactiva con Plotly
    fig = go.Figure()

    # Agregar aristas
    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='blue', width=3),
        name='Aristas'
    ))

    # Agregar nodos
    fig.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Nodos'
    ))

    # Configurar la visualización
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis=dict(title='Px', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            yaxis=dict(title='Py', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            zaxis=dict(title='Pz', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            aspectmode='data'  # Asegura proporciones correctas entre los ejes
        ),
        margin=dict(l=10, r=10, b=10, t=50),
        height=800  # Aumentar el tamaño del gráfico
    )

    fig.show()

In [6]:
def visualize_segments(df):
    # Get unique branch IDs
    unique_branches = df['Branch ID'].unique()
    colors = np.linspace(0, 1, len(unique_branches))  # Generate color indices
    color_map = dict(zip(unique_branches, colors))  # Map each branch to a color

    # Create figure
    fig = go.Figure()

    for branch_id in unique_branches:
        branch_data = df[df['Branch ID'] == branch_id]

        fig.add_trace(go.Scatter3d(
            x=branch_data['Px'],
            y=branch_data['Py'],
            z=branch_data['Pz'],
            mode='markers',
            marker=dict(size=3, color=color_map[branch_id], colorscale='viridis'),
            name=f'Branch {branch_id}'
        ))

    # Configure layout
    fig.update_layout(
        title='Segmented Coronary Artery Visualization',
        scene=dict(
            xaxis=dict(title='Px', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            yaxis=dict(title='Py', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            zaxis=dict(title='Pz', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            aspectmode='data'  # Maintain aspect ratio
        ),
        margin=dict(l=10, r=10, b=10, t=50),
        height=800  # Set figure height
    )

    fig.show()

In [7]:
def clean_dataframe(df):

    columns_to_drop = [
        'Branch ID',
        'Tx', 'Ty', 'Tz',
        'C',
        'Nx', 'Ny', 'Nz',  # Normal vector coordinates
        'BNx', 'BNy', 'BNz',  # Binormal vector coordinates
        'Dfit', 'Dmin', 'Dmax',  # Circle diameters
        'Dh', 'Xh',  # Hydraulic properties
        'Scf', 'Area', 'E'  # Sectional properties and ellipticity
    ]

    # Drop columns
    df = df.drop(columns=columns_to_drop, errors='ignore')

    # Drop duplicate rows
    df = df.drop_duplicates()

    return df

In [8]:
def calculate_mst(df):

    # Extract the coordinates of the points
    positions = df[['Px', 'Py', 'Pz']].values

    # Calculate the Euclidean distance matrix between all points
    dist_matrix = distance_matrix(positions, positions)

    # Create a graph with all nodes and distances as weights
    G = nx.Graph()
    num_nodes = len(positions)
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            G.add_edge(i, j, weight=dist_matrix[i, j])

    # Get the Minimum Spanning Tree (MST)
    MST = nx.minimum_spanning_tree(G)

    return MST

In [9]:
def visualize_mst(df, MST):

    # Extract the positions of the nodes
    positions = df[['Px', 'Py', 'Pz']].values

    # Get the positions for the nodes
    pos = {i: positions[i] for i in range(len(positions))}

    # Create lists for node and edge coordinates
    node_x, node_y, node_z = zip(*positions)

    edge_x = []
    edge_y = []
    edge_z = []
    for edge in MST.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Create the interactive visualization with Plotly
    fig = go.Figure()

    # Add edges (aristas)
    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='blue', width=2),
        name='Edges'
    ))

    # Add nodes (nodos)
    fig.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers',
        marker=dict(size=3, color='red'),
        name='Nodes'
    ))

    # Configure the layout for the visualization
    fig.update_layout(
        title='Minimum Spanning Tree (MST)',
        scene=dict(
            xaxis=dict(title='Px', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            yaxis=dict(title='Py', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            zaxis=dict(title='Pz', backgroundcolor='white', gridcolor='lightgrey', showbackground=True),
            aspectmode='data'  # Ensure correct proportions between axes
        ),
        margin=dict(l=10, r=10, b=10, t=50),
        height=800  # Increase the plot size
    )

    # Display the plot
    fig.show()

In [10]:
def label_segments(MST, df, start_node=0):
    # Identify bifurcations (nodes with degree > 2)
    bifurcations = [node for node, degree in MST.degree() if degree > 2]

    # Initialize labels for all nodes
    num_nodes = len(df)
    labels = np.zeros(num_nodes, dtype=int)
    current_label = 1

    # Traverse the MST and assign segment labels
    visited = set()
    stack = [(start_node, current_label)]

    while stack:
        node, label = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        labels[node] = label

        neighbors = list(MST.neighbors(node))
        for neighbor in neighbors:
            if neighbor not in visited:
                if neighbor in bifurcations:
                    current_label += 1  # New label at bifurcation
                    stack.append((neighbor, current_label))
                else:
                    stack.append((neighbor, label))

    # Add labels back to the dataframe
    df['Branch ID'] = labels

    return df

In [11]:
def bifurcations_labeling(MST, df, start_node=0):
    # Identify bifurcations (nodes with degree > 2)
    bifurcations = [node for node, degree in MST.degree() if degree > 2]

    # Initialize labels for all nodes
    num_nodes = len(df)
    labels = np.zeros(num_nodes, dtype=int)
    current_label = 1

    # Traverse the MST and assign segment labels
    visited = set()
    stack = [(start_node, current_label)]

    while stack:
        node, label = stack.pop()
        if node in visited:
            continue
        visited.add(node)
        labels[node] = label

        neighbors = list(MST.neighbors(node))
        unvisited_neighbors = [n for n in neighbors if n not in visited]

        if node in bifurcations:
            # Assign new labels to each branch at the bifurcation
            for neighbor in unvisited_neighbors:
                current_label += 1
                stack.append((neighbor, current_label))
        else:
            for neighbor in unvisited_neighbors:
                stack.append((neighbor, label))

    # Add labels back to the dataframe
    df['Branch ID'] = labels

    return df

## **NORMAL 1**

### **Execution - RCA Normal 1**

In [12]:
df_rca_n1 = parse_and_export_centerlines(file_path_rca_n1)
df_rca_n1.head()

,Branch ID,Px,Py,Pz,Tx,Ty,Tz,Nx,Ny,Nz,...,BNz,Dfit,Dmin,Dmax,C,Dh,Xh,Scf,Area,E
0,16,150.4411,124.7483,28.0525,-0.8410,0.5269,-0.1234,0.1021,0.3784,0.9200,...,-0.3720,0.4566,0.1851,0.7793,NaN,0.3867,0.4962,1.5989,0.1546,0.6182
1,16,149.9095,125.0840,27.9798,-0.8389,0.5338,-0.1063,0.1176,0.3686,0.9221,...,-0.3720,0.9324,0.2362,1.5503,0.0292,0.8461,0.5458,3.1222,0.6604,0.8026
2,16,149.3632,125.4346,27.9166,-0.8358,0.5421,-0.0866,0.1640,0.3971,0.9030,...,-0.4208,1.2609,0.5736,2.0474,0.0373,1.1644,0.5687,4.1918,1.2202,0.6242
3,16,148.7591,125.8320,27.8633,-0.8298,0.5549,-0.0590,0.2200,0.4225,0.8792,...,-0.4727,1.1607,0.8701,1.4998,0.0482,1.0757,0.7172,3.8404,1.0327,0.7187
4,16,148.1571,126.2426,27.8321,-0.8196,0.5723,-0.0249,0.2893,0.4510,0.8443,...,-0.5352,1.1610,0.6509,1.5376,0.0604,1.0726,0.6976,3.8314,1.0274,0.7226


In [ ]:
df_rca_n1 = clean_dataframe(df_rca_n1)
df_rca_n1

,Px,Py,Pz
0,150.4411,124.7483,28.0525
1,149.9095,125.0840,27.9798
2,149.3632,125.4346,27.9166
3,148.7591,125.8320,27.8633
4,148.1571,126.2426,27.8321
...,...,...,...
428,88.6466,103.6300,25.5747
429,88.3045,103.5971,25.8131
430,87.9935,103.5645,26.0413
431,87.7353,103.5360,26.2380


##### **Minimmum Spannig Tree Graph**

In [ ]:
MST_rca_n1 = calculate_mst(df_rca_n1)

In [ ]:
visualize_mst(df_rca_n1, MST_rca_n1)

#### **Bifurcations algotithm**

In [ ]:
df_rca_n1_labelled = bifurcations_labeling(MST_rca_n1, df_rca_n1)
df_rca_n1_labelled

,Px,Py,Pz,Branch ID
0,150.4411,124.7483,28.0525,1
1,149.9095,125.0840,27.9798,1
2,149.3632,125.4346,27.9166,1
3,148.7591,125.8320,27.8633,1
4,148.1571,126.2426,27.8321,1
...,...,...,...,...
428,88.6466,103.6300,25.5747,3
429,88.3045,103.5971,25.8131,3
430,87.9935,103.5645,26.0413,3
431,87.7353,103.5360,26.2380,3


In [ ]:
# Print unique values of the column
unique_values = df_rca_n1_labelled['Branch ID'].unique()
print(unique_values)

[1 2 3]


In [ ]:
visualize_segments(df_rca_n1_labelled)

### **Execution - LCA Normal 1**

In [ ]:
df_lca_n1 = parse_and_export_centerlines(file_path_lca_n1)
df_lca_n1.head()

,Branch ID,Px,Py,Pz,Tx,Ty,Tz,Nx,Ny,Nz,...,BNz,Dfit,Dmin,Dmax,C,Dh,Xh,Scf,Area,E
0,10,165.6963,105.4312,77.5512,-0.6097,0.2803,0.7414,-0.3427,-0.9367,0.0723,...,0.6671,0.9580,0.5374,1.4355,NaN,0.8015,0.5584,3.3391,0.6691,0.8144
1,10,165.5281,105.5010,77.7533,-0.6262,0.2326,0.7441,-0.3115,-0.9496,0.0347,...,0.6671,1.3974,0.7427,1.9252,0.1859,1.1244,0.5841,5.2718,1.4820,0.5607
2,10,165.3429,105.5610,77.9707,-0.6432,0.1762,0.7451,-0.2660,-0.9640,-0.0016,...,0.6669,1.5941,0.7662,2.1823,0.2180,1.3420,0.6150,5.7910,1.9429,0.6037
3,10,165.1206,105.6082,78.2242,-0.6623,0.0971,0.7429,-0.2025,-0.9779,-0.0528,...,0.6673,1.4368,0.9247,1.8548,0.2609,1.2641,0.6815,5.1560,1.6294,0.2421
4,10,164.8774,105.6258,78.4921,-0.6787,-0.0041,0.7344,-0.1182,-0.9863,-0.1147,...,0.6690,1.4504,0.9030,1.8502,0.3076,1.3192,0.7130,4.8825,1.6102,0.7215


In [ ]:
df_lca_n1 = clean_dataframe(df_lca_n1)
df_lca_n1

,Px,Py,Pz
0,165.6963,105.4312,77.5512
1,165.5281,105.5010,77.7533
2,165.3429,105.5610,77.9707
3,165.1206,105.6082,78.2242
4,164.8774,105.6258,78.4921
...,...,...,...
901,123.9599,130.4967,91.7940
902,123.5791,130.2197,91.8576
903,123.1971,129.9483,91.9276
904,122.8180,129.6789,92.0056


##### **Minimmum Spannig Tree Graph**

In [ ]:
MST_lca_n1 = calculate_mst(df_lca_n1)

In [ ]:
visualize_mst(df_lca_n1, MST_lca_n1)

#### **Bifurcations algotithm**

In [ ]:
df_lca_n1_labelled = bifurcations_labeling(MST_lca_n1, df_lca_n1)
df_lca_n1_labelled

,Px,Py,Pz,Branch ID
0,165.6963,105.4312,77.5512,1
1,165.5281,105.5010,77.7533,1
2,165.3429,105.5610,77.9707,1
3,165.1206,105.6082,78.2242,1
4,164.8774,105.6258,78.4921,1
...,...,...,...,...
901,123.9599,130.4967,91.7940,11
902,123.5791,130.2197,91.8576,11
903,123.1971,129.9483,91.9276,11
904,122.8180,129.6789,92.0056,11


In [ ]:
# Print unique values of the column
unique_values = df_lca_n1_labelled['Branch ID'].unique()
print(unique_values)

[ 1  9 10  2  3  4  7  5  6  8 11]


In [ ]:
visualize_segments(df_lca_n1_labelled)

## **NORMAL 7**

### **Execution - RCA Normal 7**

In [ ]:
df_rca_n7 = parse_and_export_centerlines(file_path_rca_n7)
df_rca_n7.head()

,Branch ID,Px,Py,Pz,Tx,Ty,Tz,Nx,Ny,Nz,...,BNz,Dfit,Dmin,Dmax,C,Dh,Xh,Scf,Area,E
0,1,85.5587,89.4809,88.5482,-0.9627,-0.2567,-0.0857,0.2335,-0.9480,0.2164,...,0.9725,1.3918,0.9133,2.1490,NaN,1.2213,0.5683,4.7051,1.4366,0.8269
1,1,85.0071,89.3207,88.5016,-0.9512,-0.2990,-0.0759,0.2765,-0.9355,0.2200,...,0.9725,2.0776,1.3993,2.8398,0.0780,1.8308,0.6447,7.0335,3.2193,0.7766
2,1,84.4266,89.1223,88.4583,-0.9350,-0.3488,-0.0644,0.3290,-0.9208,0.2095,...,0.9757,2.9739,1.9300,4.7210,0.0963,2.3312,0.4938,10.5551,6.1514,0.8334
3,1,83.7701,88.8501,88.4178,-0.9064,-0.4197,-0.0486,0.4032,-0.8936,0.1974,...,0.9791,3.5464,2.5297,5.0184,0.1232,3.1165,0.6210,11.9780,9.3325,0.7973
4,1,83.1066,88.5028,88.3885,-0.8586,-0.5118,-0.0288,0.4993,-0.8476,0.1795,...,0.9833,4.0744,3.2805,5.1569,0.1592,3.9189,0.7599,13.2049,12.9372,0.5225


In [ ]:
df_rca_n7 = clean_dataframe(df_rca_n7)
df_rca_n7

,Px,Py,Pz
0,85.5587,89.4809,88.5482
1,85.0071,89.3207,88.5016
2,84.4266,89.1223,88.4583
3,83.7701,88.8501,88.4178
4,83.1066,88.5028,88.3885
...,...,...,...
312,68.0813,88.6793,29.1486
313,67.8826,88.3532,28.9623
314,67.7027,88.0556,28.7639
315,67.5542,87.8073,28.5812


##### **Minimmum Spannig Tree Graph**

In [ ]:
MST_rca_n7 = calculate_mst(df_rca_n7)

In [ ]:
visualize_mst(df_rca_n7, MST_rca_n7)

#### **Bifurcations algotithm**

In [ ]:
df_rca_n7_labelled = bifurcations_labeling(MST_rca_n7, df_rca_n7)
df_rca_n7_labelled

,Px,Py,Pz,Branch ID
0,85.5587,89.4809,88.5482,1
1,85.0071,89.3207,88.5016,1
2,84.4266,89.1223,88.4583,1
3,83.7701,88.8501,88.4178,1
4,83.1066,88.5028,88.3885,1
...,...,...,...,...
312,68.0813,88.6793,29.1486,2
313,67.8826,88.3532,28.9623,2
314,67.7027,88.0556,28.7639,2
315,67.5542,87.8073,28.5812,2


In [ ]:
# Print unique values of the column
unique_values = df_rca_n7_labelled['Branch ID'].unique()
print(unique_values)

[1 3 2]


In [ ]:
visualize_segments(df_rca_n7_labelled)

### **Execution - LCA Normal 7**

In [ ]:
df_lca_n7 = parse_and_export_centerlines(file_path_lca_n7)
df_lca_n7.head()

,Branch ID,Px,Py,Pz,Tx,Ty,Tz,Nx,Ny,Nz,...,BNz,Dfit,Dmin,Dmax,C,Dh,Xh,Scf,Area,E
0,0,108.2599,119.5730,91.2165,0.9665,-0.2565,-0.0073,0.1838,0.7119,-0.6778,...,0.7352,4.0877,3.5013,6.0208,NaN,3.8351,0.6370,13.3763,12.8247,0.6743
1,0,108.5385,119.5187,91.1969,0.9838,-0.1225,-0.1307,0.0046,0.7467,-0.6651,...,0.7352,6.6115,3.3573,11.1789,0.6499,5.2093,0.4660,23.1192,30.1087,0.9279
2,0,108.7702,119.5018,91.1551,0.9759,-0.0337,-0.2155,-0.1377,0.6709,-0.7287,...,0.6501,5.7747,3.3146,10.1031,0.3994,4.8978,0.4848,19.6368,24.0441,0.8955
3,0,109.0604,119.4995,91.0814,0.9680,-0.0106,-0.2508,0.0993,-0.9016,0.4211,...,-0.8717,5.5881,3.0409,10.4102,0.1263,4.8086,0.4619,18.9056,22.7275,0.8829
4,0,109.4093,119.4893,90.9940,0.9703,-0.0404,-0.2387,0.0225,-0.9666,0.2553,...,-0.9370,5.7847,2.5912,12.1272,0.0537,4.8027,0.3960,19.9101,23.9056,0.8908


In [ ]:
df_lca_n7 = clean_dataframe(df_lca_n7)
df_lca_n7

,Px,Py,Pz
0,108.2599,119.5730,91.2165
1,108.5385,119.5187,91.1969
2,108.7702,119.5018,91.1551
3,109.0604,119.4995,91.0814
4,109.4093,119.4893,90.9940
...,...,...,...
841,97.7172,143.5526,48.9518
842,97.5046,143.3270,48.4060
843,97.3041,143.1103,47.9115
844,97.1332,142.9231,47.5020


##### **Minimmum Spannig Tree Graph**

In [ ]:
MST_lca_n7 = calculate_mst(df_lca_n7)

In [ ]:
visualize_mst(df_lca_n7, MST_lca_n7)

#### **Bifurcations algotithm**

In [ ]:
df_lca_n7_labelled = bifurcations_labeling(MST_lca_n7, df_lca_n7)
df_lca_n7_labelled

,Px,Py,Pz,Branch ID
0,108.2599,119.5730,91.2165,1
1,108.5385,119.5187,91.1969,1
2,108.7702,119.5018,91.1551,8
3,109.0604,119.4995,91.0814,8
4,109.4093,119.4893,90.9940,8
...,...,...,...,...
841,97.7172,143.5526,48.9518,10
842,97.5046,143.3270,48.4060,10
843,97.3041,143.1103,47.9115,10
844,97.1332,142.9231,47.5020,10


In [ ]:
# Print unique values of the column
unique_values = df_lca_n7_labelled['Branch ID'].unique()
print(unique_values)

[ 1  8  9 11  5  6  2  7  3  4 10]


In [ ]:
visualize_segments(df_lca_n7_labelled)

## **DISEASED 7**

### **Execution - RCA Diseased 7**

In [ ]:
df_rca_d7 = parse_and_export_centerlines(file_path_rca_d7)
df_rca_d7.head()

,Branch ID,Px,Py,Pz,Tx,Ty,Tz,Nx,Ny,Nz,...,BNz,Dfit,Dmin,Dmax,C,Dh,Xh,Scf,Area,E
0,0,142.4291,101.9354,38.8485,-0.8962,0.4216,-0.1378,0.2698,0.7648,0.5850,...,-0.7992,0.7898,0.4322,1.0254,NaN,0.6824,0.6655,2.7905,0.4760,0.6980
1,0,141.9711,102.1537,38.7798,-0.8928,0.4311,-0.1306,0.2809,0.7595,0.5867,...,-0.7992,0.7911,0.3724,1.2606,0.0242,0.6898,0.5472,2.8340,0.4887,0.3149
2,0,141.4801,102.3940,38.7101,-0.8891,0.4411,-0.1218,0.2539,0.6969,0.6707,...,-0.7316,1.0602,0.5492,2.0556,0.0261,0.9314,0.4531,3.6030,0.8390,0.7985
3,0,140.9089,102.6818,38.6355,-0.8851,0.4525,-0.1090,0.2081,0.5942,0.7770,...,-0.6200,1.2583,0.9344,2.1455,0.0287,1.0505,0.4896,4.4687,1.1736,0.7550
4,0,140.3051,102.9948,38.5666,-0.8818,0.4626,-0.0915,0.1104,0.3911,0.9137,...,-0.3959,1.4088,1.2336,1.7459,0.0317,1.3479,0.7721,4.5750,1.5417,0.4940


In [ ]:
df_rca_d7 = clean_dataframe(df_rca_d7)
df_rca_d7

,Px,Py,Pz
0,142.4291,101.9354,38.8485
1,141.9711,102.1537,38.7798
2,141.4801,102.3940,38.7101
3,140.9089,102.6818,38.6355
4,140.3051,102.9948,38.5666
...,...,...,...
401,108.0949,89.3446,34.9859
402,108.3885,88.9726,34.9874
403,108.6563,88.6091,34.9921
404,108.8796,88.2897,34.9986


##### **Minimmum Spannig Tree Graph**

In [ ]:
MST_rca_d7 = calculate_mst(df_rca_d7)

In [ ]:
visualize_mst(df_rca_d7, MST_rca_d7)

#### **Bifurcations algotithm**

In [ ]:
df_rca_d7_labelled = bifurcations_labeling(MST_rca_d7, df_rca_d7)
df_rca_d7_labelled

,Px,Py,Pz,Branch ID
0,142.4291,101.9354,38.8485,1
1,141.9711,102.1537,38.7798,1
2,141.4801,102.3940,38.7101,1
3,140.9089,102.6818,38.6355,1
4,140.3051,102.9948,38.5666,1
...,...,...,...,...
401,108.0949,89.3446,34.9859,2
402,108.3885,88.9726,34.9874,2
403,108.6563,88.6091,34.9921,2
404,108.8796,88.2897,34.9986,2


In [ ]:
# Print unique values of the column
unique_values = df_rca_d7_labelled['Branch ID'].unique()
print(unique_values)

[1 3 2]


In [ ]:
visualize_segments(df_rca_d7_labelled)

### **Execution - LCA Diseased 7**

In [ ]:
df_lca_d7 = parse_and_export_centerlines(file_path_lca_d7)
df_lca_d7.head()

,Branch ID,Px,Py,Pz,Tx,Ty,Tz,Nx,Ny,Nz,...,BNz,Dfit,Dmin,Dmax,C,Dh,Xh,Scf,Area,E
0,1,117.5563,103.9986,110.8087,-0.9449,-0.2024,-0.2572,-0.2978,0.2060,0.9321,...,-0.2549,5.1939,3.4039,9.1903,NaN,4.0982,0.4459,19.0982,19.5668,0.8484
1,1,115.5842,103.5882,110.3172,-0.9569,-0.1933,-0.2168,-0.2569,0.2145,0.9423,...,-0.2549,4.3008,3.7126,5.0639,0.0207,3.9899,0.7879,14.4542,14.4177,0.5552
2,1,113.6132,103.1759,109.8462,-0.9267,-0.2260,-0.3002,0.3605,-0.3100,-0.8797,...,0.3687,4.2543,3.5637,5.0379,0.1121,3.7533,0.7450,15.0404,14.1127,0.6047
3,1,111.8039,102.6826,109.0206,-0.8331,-0.2137,-0.5101,0.4144,0.3696,-0.8317,...,-0.2194,5.2090,4.4574,6.1512,0.1285,4.5990,0.7476,18.4221,21.1806,0.4395
4,1,110.2861,102.3578,107.8257,-0.7061,-0.1165,-0.6984,0.6060,0.4107,-0.6812,...,-0.2194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_lca_d7 = clean_dataframe(df_lca_d7)
df_lca_d7

,Px,Py,Pz
0,117.5563,103.9986,110.8087
1,115.5842,103.5882,110.3172
2,113.6132,103.1759,109.8462
3,111.8039,102.6826,109.0206
4,110.2861,102.3578,107.8257
...,...,...,...
606,154.6442,67.3693,87.4326
607,154.7963,67.1634,87.2352
608,154.9400,66.9529,87.0837
609,155.0642,66.7602,86.9784


##### **Minimmum Spannig Tree Graph**

In [ ]:
MST_lca_d7 = calculate_mst(df_lca_d7)

In [ ]:
visualize_mst(df_lca_d7, MST_lca_d7)

#### **Bifurcations algotithm**

In [ ]:
df_lca_d7_labelled = bifurcations_labeling(MST_lca_d7, df_lca_d7)
df_lca_d7_labelled

,Px,Py,Pz,Branch ID
0,117.5563,103.9986,110.8087,1
1,115.5842,103.5882,110.3172,4
2,113.6132,103.1759,109.8462,4
3,111.8039,102.6826,109.0206,4
4,110.2861,102.3578,107.8257,4
...,...,...,...,...
606,154.6442,67.3693,87.4326,7
607,154.7963,67.1634,87.2352,7
608,154.9400,66.9529,87.0837,7
609,155.0642,66.7602,86.9784,7


In [ ]:
# Print unique values of the column
unique_values = df_lca_d7_labelled['Branch ID'].unique()
print(unique_values)

[1 4 8 6 5 3 2 7]


In [ ]:
visualize_segments(df_lca_d7_labelled)

## **DISEASED 9**

### **Execution - RCA Diseased 9**

In [ ]:
df_rca_d9 = parse_and_export_centerlines(file_path_rca_d9)
df_rca_d9.head()

,Branch ID,Px,Py,Pz,Tx,Ty,Tz,Nx,Ny,Nz,...,BNz,Dfit,Dmin,Dmax,C,Dh,Xh,Scf,Area,E
0,4,93.2438,127.3554,32.7727,-0.2171,0.9538,0.2076,0.9662,0.1797,0.1848,...,-0.9606,3.5249,2.4634,5.9066,NaN,2.8607,0.4843,14.0066,10.0171,0.7079
1,4,93.1850,127.7816,32.8710,-0.0474,0.9705,0.2364,0.9892,0.0127,0.1463,...,-0.9606,4.3220,2.6934,7.9420,0.3944,2.8088,0.3537,18.7730,13.1823,0.8406
2,4,93.1918,128.1071,32.9542,0.0921,0.9608,0.2615,0.9688,-0.1471,0.1992,...,-0.9444,NaN,2.3501,26.0578,0.4566,2.5180,0.0966,39.3249,24.7552,0.9783
3,4,93.2544,128.4827,33.0637,0.1814,0.9408,0.2863,0.2653,-0.3272,0.9070,...,-0.3090,3.0845,1.1590,6.5214,0.0440,1.8638,0.2858,15.6600,7.2969,0.8409
4,4,93.3431,128.9215,33.2028,0.2070,0.9283,0.3090,0.8134,-0.3388,0.4728,...,-0.8252,2.3993,1.6105,3.1766,0.1194,2.1849,0.6878,8.1158,4.4330,0.5677


In [ ]:
df_rca_d9 = clean_dataframe(df_rca_d9)
df_rca_d9

,Px,Py,Pz
0,93.2438,127.3554,32.7727
1,93.1850,127.7816,32.8710
2,93.1918,128.1071,32.9542
3,93.2544,128.4827,33.0637
4,93.3431,128.9215,33.2028
...,...,...,...
504,89.5781,111.5224,113.7899
505,89.6284,112.0969,113.8226
506,89.8562,112.6161,113.6975
507,90.0840,113.1352,113.5725


##### **Minimmum Spannig Tree Graph**

In [ ]:
MST_rca_d9 = calculate_mst(df_rca_d9)

In [ ]:
visualize_mst(df_rca_d9, MST_rca_d9)

#### **Bifurcations algotithm**

In [ ]:
df_rca_d9_labelled = bifurcations_labeling(MST_rca_d9, df_rca_d9)
df_rca_d9_labelled

,Px,Py,Pz,Branch ID
0,93.2438,127.3554,32.7727,1
1,93.1850,127.7816,32.8710,2
2,93.1918,128.1071,32.9542,2
3,93.2544,128.4827,33.0637,2
4,93.3431,128.9215,33.2028,2
...,...,...,...,...
504,89.5781,111.5224,113.7899,3
505,89.6284,112.0969,113.8226,3
506,89.8562,112.6161,113.6975,3
507,90.0840,113.1352,113.5725,3


In [ ]:
# Print unique values of the column
unique_values = df_rca_d9_labelled['Branch ID'].unique()
print(unique_values)

[1 2 4 3]


In [ ]:
visualize_segments(df_rca_d9_labelled)

### **Execution - LCA Diseased 9**

In [ ]:
df_lca_d9 = parse_and_export_centerlines(file_path_lca_d9)
df_lca_d9.head()

,Branch ID,Px,Py,Pz,Tx,Ty,Tz,Nx,Ny,Nz,...,BNz,Dfit,Dmin,Dmax,C,Dh,Xh,Scf,Area,E
0,1,163.5610,89.3273,18.9400,-0.2415,-0.7361,0.6323,-0.5056,-0.4607,-0.7294,...,-0.2609,1.5926,1.1009,2.0319,NaN,1.4475,0.7124,5.3520,1.9368,0.6019
1,1,163.4660,89.0403,19.1837,-0.2481,-0.7421,0.6227,-0.5024,-0.4511,-0.7376,...,-0.2609,1.5868,1.1144,2.0671,0.0337,1.5196,0.7351,5.1797,1.9678,0.3133
2,1,163.3584,88.7218,19.4474,-0.2551,-0.7493,0.6111,-0.4171,-0.4848,-0.7687,...,-0.1889,1.6057,1.1331,2.0896,0.0387,1.5351,0.7346,5.2362,2.0095,0.4792
3,1,163.2258,88.3356,19.7560,-0.2630,-0.7603,0.5939,-0.3134,-0.5149,-0.7979,...,-0.1028,1.5153,0.8816,2.1114,0.0468,1.3753,0.6513,5.0753,1.7450,0.7619
4,1,163.0765,87.9063,20.0819,-0.2701,-0.7761,0.5699,-0.1750,-0.5424,-0.8217,...,0.0106,1.4744,0.9599,1.8733,0.0593,1.2950,0.6913,5.1248,1.6592,0.5440


In [ ]:
df_lca_d9 = clean_dataframe(df_lca_d9)
df_lca_d9

,Px,Py,Pz
0,163.5610,89.3273,18.9400
1,163.4660,89.0403,19.1837
2,163.3584,88.7218,19.4474
3,163.2258,88.3356,19.7560
4,163.0765,87.9063,20.0819
...,...,...,...
770,151.9016,162.3063,72.1131
771,151.8617,162.3913,71.5844
772,151.8363,162.4205,71.0832
773,151.8230,162.4104,70.6496


##### **Minimmum Spannig Tree Graph**

In [ ]:
MST_lca_d9 = calculate_mst(df_lca_d9)

In [ ]:
visualize_mst(df_lca_d9, MST_lca_d9)

#### **Bifurcations algotithm**

In [ ]:
df_lca_d9_labelled = bifurcations_labeling(MST_lca_d9, df_lca_d9)
df_lca_d9_labelled

,Px,Py,Pz,Branch ID
0,163.5610,89.3273,18.9400,1
1,163.4660,89.0403,19.1837,1
2,163.3584,88.7218,19.4474,1
3,163.2258,88.3356,19.7560,1
4,163.0765,87.9063,20.0819,1
...,...,...,...,...
770,151.9016,162.3063,72.1131,9
771,151.8617,162.3913,71.5844,9
772,151.8363,162.4205,71.0832,9
773,151.8230,162.4104,70.6496,9


In [ ]:
# Print unique values of the column
unique_values = df_lca_d9_labelled['Branch ID'].unique()
print(unique_values)

[1 2 5 6 4 3 8 7 9]


In [ ]:
visualize_segments(df_lca_d9_labelled)